In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")

!git clone https://{token}@github.com/Benedart/RecSys-2022-Challenge-Polimi.git

Cloning into 'RecSys-2022-Challenge-Polimi'...
remote: Enumerating objects: 608, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 608 (delta 75), reused 159 (delta 69), pack-reused 443
Receiving objects: 100% (608/608), 1.06 GiB | 44.68 MiB/s, done.
Resolving deltas: 100% (246/246), done.
Updating files: 100% (205/205), done.


In [2]:
import os

os.chdir( "./RecSys-2022-Challenge-Polimi")

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps
import csv
from tqdm import tqdm

from Recommenders.Recommender_import_list import *

In [4]:
URM_all = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/interactions_and_impressions.csv")
ICM_type = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_type.csv")
ICM_length = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_length.csv")

ICM_all = pd.concat([ICM_type, ICM_length]).sort_values(by='item_id')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
mapped_id, original_id = pd.factorize(URM_all["UserID"].unique())
user_original_Id_to_index = pd.Series(mapped_id, index=original_id)

print("Unique UserID in the URM are {}".format(len(original_id)))

mapped_id, original_id = pd.factorize(URM_all["ItemID"].unique())

print("Unique ItemID in the URM are {}".format(len(original_id)))

all_item_indices = pd.concat([URM_all["ItemID"], ICM_all["item_id"]], ignore_index=True)
mapped_id, original_id = pd.factorize(all_item_indices.unique())

print("Unique ItemID in the URM and ICM are {}".format(len(original_id)))

item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(ICM_all["feature_id"].unique())
feature_original_ID_to_index = pd.Series(mapped_id, index=original_id)

print("Unique FeatureID in the URM are {}".format(len(feature_original_ID_to_index)))

URM_all["UserID"] = URM_all["UserID"].map(user_original_Id_to_index)
URM_all["ItemID"] = URM_all["ItemID"].map(item_original_ID_to_index)
ICM_all["item_id"] = ICM_all["item_id"].map(item_original_ID_to_index)
ICM_all["feature_id"] = ICM_all["feature_id"].map(feature_original_ID_to_index)

Unique UserID in the URM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 27968
Unique FeatureID in the URM are 6


In [6]:
URM_all = URM_all.drop(["Impressions", "Data"], axis=1).drop_duplicates()
URM_all["Rating"] = 1

In [7]:
n_users = len(user_original_Id_to_index)
n_items = len(item_original_ID_to_index)
n_features = len(feature_original_ID_to_index)

from sklearn.model_selection import train_test_split

seed = 42

(user_ids_training_validation, user_ids_test,
 item_ids_training_validation, item_ids_test,
 ratings_training_validation, ratings_test) = train_test_split(URM_all.UserID,
                                                    URM_all.ItemID,
                                                    URM_all.Rating,
                                                    test_size=0.15,
                                                    shuffle=True,
                                                    random_state=seed)

(user_ids_training, user_ids_validation,
 item_ids_training, item_ids_validation,
 ratings_training, ratings_validation) = train_test_split(user_ids_training_validation,
                                                          item_ids_training_validation,
                                                          ratings_training_validation,
                                                          test_size=0.15,
                                                          shuffle=True,
                                                          random_state=seed)

URM_all = sps.csr_matrix((URM_all.Rating, (URM_all.UserID, URM_all.ItemID)),
                           shape = (n_users, n_items))

URM_train = sps.csr_matrix((ratings_training, (user_ids_training, item_ids_training)),
                           shape = (n_users, n_items))

URM_validation = sps.csr_matrix((ratings_validation, (user_ids_validation, item_ids_validation)),
                           shape = (n_users, n_items))

URM_train_validation = sps.csr_matrix((ratings_training_validation, (user_ids_training_validation, item_ids_training_validation)),
                           shape = (n_users, n_items))

URM_test = sps.csr_matrix((ratings_test, (user_ids_test, item_ids_test)),
                           shape = (n_users, n_items))


ICM_all = sps.csr_matrix((np.ones(len(ICM_all["item_id"].values)), 
                          (ICM_all["item_id"].values, ICM_all["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all.data = np.ones_like(ICM_all.data)

ICM_all = sps.csr_matrix(ICM_all)
features_per_item = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csc_matrix(ICM_all)
items_per_feature = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csr_matrix(ICM_all)

In [8]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1520 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 976 ( 2.3%) Users that have less than 1 test interactions


## Insert model here

In [9]:
model = FeatureCombinedImplicitALSRecommender
GPU = True

In [10]:
import optuna as op

def objective(trial):
    iterations = trial.suggest_int("iterations", 100, 300)
    factors = trial.suggest_int("factors", 1, 150)
    regularization = trial.suggest_float("regularization", 0.001, 0.1)
    urm_alpha = trial.suggest_float("urm_alpha", 2, 4)
    icm_alpha = trial.suggest_float("icm_alpha", 0, 1)
    use_gpu = trial.suggest_categorical("use_gpu", [GPU])
    
    recommender = model(URM_train, ICM_all)
    recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [11]:
best_params = {
    'iterations': 120,
    'factors': 80,
    'regularization': 0.04590878515116987,
    'urm_alpha': 2.9974929864429285,
    'icm_alpha': 0.8631236369566112,
    'use_gpu': True
}

study = op.create_study(direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=500)

[I 2023-01-10 11:06:36,846] A new study created in memory with name: no-name-dbdf56a1-d1e7-454f-aef7-244a323f8b36


EvaluatorHoldout: Processed 40109 (100.0%) in 33.48 sec. Users per second: 1198


[I 2023-01-10 11:07:32,978] Trial 0 finished with value: 0.014338267506758554 and parameters: {'iterations': 120, 'factors': 80, 'regularization': 0.04590878515116987, 'urm_alpha': 2.9974929864429285, 'icm_alpha': 0.8631236369566112, 'use_gpu': True}. Best is trial 0 with value: 0.014338267506758554.


EvaluatorHoldout: Processed 40109 (100.0%) in 34.00 sec. Users per second: 1180


[I 2023-01-10 11:08:50,893] Trial 1 finished with value: 0.014525427139616109 and parameters: {'iterations': 260, 'factors': 111, 'regularization': 0.02580105613949528, 'urm_alpha': 3.7317507596661645, 'icm_alpha': 0.4606070093301988, 'use_gpu': True}. Best is trial 1 with value: 0.014525427139616109.


...

[I 2023-01-10 20:24:37,812] Trial 497 finished with value: 0.014442266846652462 and parameters: {'iterations': 143, 'factors': 31, 'regularization': 0.04413584647602702, 'urm_alpha': 3.895430113883065, 'icm_alpha': 0.27704730625699076, 'use_gpu': True}. Best is trial 297 with value: 0.014713745321776026.


EvaluatorHoldout: Processed 40109 (100.0%) in 33.46 sec. Users per second: 1199


[I 2023-01-10 20:25:35,421] Trial 498 finished with value: 0.014642048829637704 and parameters: {'iterations': 151, 'factors': 93, 'regularization': 0.04771381475825463, 'urm_alpha': 3.96205219326598, 'icm_alpha': 0.1574610149192216, 'use_gpu': True}. Best is trial 297 with value: 0.014713745321776026.


EvaluatorHoldout: Processed 40109 (100.0%) in 32.98 sec. Users per second: 1216


[I 2023-01-10 20:26:31,584] Trial 499 finished with value: 0.014527032883012997 and parameters: {'iterations': 145, 'factors': 90, 'regularization': 0.05121678913797842, 'urm_alpha': 3.9060482219996238, 'icm_alpha': 0.22715364401295698, 'use_gpu': True}. Best is trial 297 with value: 0.014713745321776026.


In [12]:
study.best_params

{'iterations': 154,
 'factors': 87,
 'regularization': 0.027737993520896242,
 'urm_alpha': 3.9693090150756634,
 'icm_alpha': 0.6816896641474052,
 'use_gpu': True}

In [13]:
final = model(URM_train_validation, ICM_all)
final.fit(**study.best_params)

In [14]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 976 ( 2.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 40653 (100.0%) in 34.25 sec. Users per second: 1187


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.042491                 0.070862  0.066381  0.019855    0.032954   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.141788  0.067582  0.051815  0.300421       0.16692  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976555          0.293377    0.976555       0.012823   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.921933                   0.997523             0.028769   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   